In [1]:
%cd ..
%load_ext nb_black

/home/hardianlawi/attention-lstm-sentiment-analysis


<IPython.core.display.Javascript object>

In [2]:
import numpy as np
import tensorflow as tf
from src.preprocess import Preprocessor
from src.data import get_data, ID2WORD, OOV_ID
from src.models import get_model


vocabulary_size = 5000
embedding_size = 32
max_words = 500

(X_train, y_train), (X_test, y_test) = get_data(vocabulary_size)

print("Maximum review length: {}".format(len(max((X_train + X_test), key=len))))
print("Minimum review length: {}".format(len(min((X_train + X_test), key=len))))


preprocessor = Preprocessor(maxlen=max_words, oov_token=ID2WORD[OOV_ID])
preprocessor.fit_on_texts(X_train + X_test)

X_train = preprocessor.transform(X_train)
X_test = preprocessor.transform(X_test)


Maximum review length: 2494
Minimum review length: 7


<IPython.core.display.Javascript object>

In [3]:
model = get_model("lstm", max_words, vocabulary_size, embedding_size)
model.summary(line_length=200)
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

Model: "model"
________________________________________________________________________________________________________________________________________________________________________________________________________
Layer (type)                                                                              Output Shape                                                                    Param #                       
input_1 (InputLayer)                                                                      [(None, 500)]                                                                   0                             
________________________________________________________________________________________________________________________________________________________________________________________________________
embedding (Embedding)                                                                     (None, 500, 32)                                                                 160000     

<IPython.core.display.Javascript object>

In [4]:
batch_size = 64
num_epochs = 3
X_valid, y_valid = X_train[:batch_size], y_train[:batch_size]
X_train2, y_train2 = X_train[batch_size:], y_train[batch_size:]

model.fit(
    X_train2,
    y_train2,
    validation_data=(X_valid, y_valid),
    batch_size=batch_size,
    epochs=num_epochs,
)

Train on 24936 samples, validate on 64 samples
Epoch 1/3
24936/24936 [==============================] - 268s 11ms/sample - loss: 0.5368 - accuracy: 0.7224 - val_loss: 0.2650 - val_accuracy: 0.9375
Epoch 2/3
24936/24936 [==============================] - 233s 9ms/sample - loss: 0.3966 - accuracy: 0.8256 - val_loss: 0.3048 - val_accuracy: 0.8906
Epoch 3/3
24936/24936 [==============================] - 219s 9ms/sample - loss: 0.2791 - accuracy: 0.8896 - val_loss: 0.2118 - val_accuracy: 0.9219


<IPython.core.display.Javascript object>

In [5]:
scores = model.evaluate(X_test, y_test, verbose=0)
print("Test accuracy:", scores[1])

Test accuracy: 0.87028


<IPython.core.display.Javascript object>